voici un lien explicatiffe [pour le travaille :)](https://youtu.be/dQw4w9WgXcQ)
non je rigole XD bref de plaisenterie.

# Scraping d'allocine

![image de presentation de scraping](https://files.realpython.com/media/Build-a-Web-Scraper-With-Requests-and-Beautiful-Soup_Watermarked.37918fb3906c.jpg)

## Membre du Projet
| Nom   / surnom        | Rôle         |
|----------------|----------------|
| Yassine Zaoui / yatsu     | Développeur   |
| Shiek / le petit scientifique   | Développeur   |

# BUT :

1. recuperer les de la page d'allocine contenant les fim dans cette [page](https://www.allocine.fr/films/) 
    recuperer :
    0. l'ID
    1. le titre
    2. Date de sortie
    3. Duree du film
    4. Type (fantastique, aventure , action)
    5. Realisateur
    6. Acteur
    7. Synopsys
    8. Note Presse
    9. Note Spectateur
    10. Affiche du film
    11. l'url
2. metre les information dans un csv
3. puis dans une base de donne dans une vm
4. faire des graphique de :
        
        - le classement des meilleur categorie de film en fonction des note Presse / publique
        - Meilleur film en fonctione des Note pour chaque anné

# Structure du rangement des dossier et fichier

- le dossier test contient plusieur petit test de programme / fonction pour voir ce qu'on le garde au final
___

### les Bibliotheque importer son :

In [30]:
import requests
import time
import csv
from bs4 import BeautifulSoup

- "**request**" permet de faire des requette a la page
- "**beautifulSoup**" Permet de manipuler des fichier html ou xml
- "**time**" Permet de faire un temp de pause pour la recolte des donne
- "**csv**" Permet de cree et manipuler les fichier csv


---
#### Fonction manipulation de string a utilise pour plus tard :

In [31]:
######- manipulation de string pour recuperer les dates time et les categories -#############
def extract_time_and_categories(text):
    parts = text.split("\n")
    date = parts[1].strip()
    time = parts[3].strip()
    categories = parts[5:]
    return date, time, categories
######- manipulation de string pour recuperer les acteurs -#################################
def extract_Acteurs(text):
    text = text[5:]
    realisateur = ""
    for i in text :
        if (i != "\n"):
            realisateur += i
        else :
            realisateur += " "
    return realisateur[: -1]
############################################################################################

---
# Etape 1 Scrapper :

Cree une fonction qui me retourne un tab d'un tableau content les infomation shouaite 

# Constante

In [32]:
#l' url du lien que je veux scraper
url = 'https://www.allocine.fr/films/'
#je recupere la page et elle est en requeste
response = requests.get(url)

#soup = au body de ma page interprete en html
soup = BeautifulSoup(response.text, 'html.parser')

## Code Scrap

In [33]:
############################################################- 1
def get_all_Note(presentation):
    notes = presentation.find_all("span", "stareval-note")
    if len(notes) == 0:
        return "NULL", "NULL"
    elif len(notes) == 1:
        return notes[0].text, "NULL"
    return notes[0].text, notes[1].text
############################################################- 2
def get_all_Acteur(presentation):
    acteurs = presentation.find("div", "meta-body-item meta-body-actor")
    if acteurs is None:
        return ""
    acteurs = acteurs.text
    return extract_Acteurs(acteurs)
############################################################- 3
def get_all_realisateurs(presentation):
    realisateursBalise = presentation.find("div", "meta-body-item meta-body-direction")
    if realisateursBalise is None:
        return "NULL"
    realisateursSpan = realisateursBalise.find_all("span")[1:]
    realisateursA = realisateursBalise.find_all("a")
    realisateurs = ""
    for nom in realisateursA:
        realisateurs += nom.text + ", "
    for nom in realisateursSpan:
        realisateurs += nom.text + ", "
    return realisateurs[: -2]
############################################################- 4
def get_all_date_time_categories(presentation):
    div = presentation.find("div", "meta-body-item meta-body-info").text
    date, time, categories = extract_time_and_categories(div)
    categories = " ".join(categories)
    return date, time, categories
###########################################################- 5
def get_photo(presentation):
    photo = presentation.find("img", "thumbnail-img")
    key = photo.get("data-src")
    if key is not None:
        photo = presentation.find("img", "thumbnail-img")['data-src']
    else :
        photo = presentation.find("img", "thumbnail-img")['src']
    return photo
###########################################################- 6
def get_synopsys(presentation):
    synopsys = presentation.find("div", "content-txt")
    if synopsys is None:
        return ""
    synopsys = synopsys.text
    synopsys = synopsys[: -1]
    return synopsys
###########################################################- 7
def affichage_data(i, titre, photo, date, time, categories, realisateurs, acteurs, synopsys, notePresse, notePublique, url, tab):
    if tab is not None:
        print("URL = "+ tab[-1], "\nID = " + tab[0],"\ntitre = " + tab[1], "\nimage = " + tab[2], "\ndate = " + tab[3], "\ntime = " + tab[4], "\ncategorie = " + tab[5])
        print("Realisateur = " + tab[6], "\nActeur = " + tab[7], "\nSynopsys = " + tab[8], "\nNotePresse = " + tab[9])
        print("NotePublique = " + tab[10], "\n-------------------")
        return
    print("URL = "+ url, "\nID = " + str(i),"\ntitre = " + titre, "\nimage = " + photo, "\ndate = " + date, "\ntime = " + time, "\ncategorie = " + categories)
    print("Realisateur = " + realisateurs, "\nActeur = " + acteurs, "\nSynopsys = " + synopsys, "\nNotePresse = " + notePresse)
    print("NotePublique = " + notePublique, "\n-------------------")
##########################################################- 8
def get_next_page(nbr_of_page):
    if nbr_of_page > 7575:
        return None
    new_url = 'https://www.allocine.fr/films/' + "?page=" + str(nbr_of_page)
    response = requests.get(new_url)
    page =BeautifulSoup(response.text, 'html.parser')
    return page, new_url
#########################################################- 9
def get_all_presentation_of_this_page(page):
    all_presentations_of_this_page = page.find_all("div", class_="card entity-card entity-card-list cf")
    #tout les elment que je shouaite scrap on tousse la meme structure et son tout dans une dic qui a 
    #comme class card entity-card entity-card-list cf
    return all_presentations_of_this_page
###########################################################- 7
def transform_data_into_tab(ID, titre, photo, date, time, categories, realisateurs, acteurs, synopsys, notePresse, notePublique, url):
    return [[str(ID), titre, photo, date, time, categories, realisateurs, acteurs, synopsys, notePresse, notePublique, url]]
############################################################- Tout avoir

def get_all_data_movie(all_presentations_of_this_page, ID, nbr_of_movie, url, affichage):
    tab = []
    for presentation in all_presentations_of_this_page:
        notePresse, notePublique = get_all_Note(presentation)
        ###########################################################
        synopsys = get_synopsys(presentation)
        ###########################################################
        acteurs = get_all_Acteur(presentation)
        ###########################################################
        realisateurs = get_all_realisateurs(presentation)
        ###########################################################
        date, time, categories = get_all_date_time_categories(presentation)
        ###########################################################
        photo = get_photo(presentation)
        ###########################################################
        titre = presentation.find("a", "meta-title-link").text
        ###########################################################
        tab = tab + transform_data_into_tab(ID, titre, photo, date, time, categories, realisateurs, acteurs, synopsys, notePresse, notePublique, url)
        if affichage:
            affichage_data(ID, titre, photo, date, time, categories, realisateurs, acteurs, synopsys, notePresse, notePublique, url)
        if nbr_of_movie <= ID:
            return -1, tab
        ###########################################################
        ID = ID + 1
    return ID, tab
############################################################- Combien de page

def get_all_data_from_movie(nbr_of_movie):
    ID = 0
    nbr_of_page = 1
    all_tab =[["Id", "titre", "affiche","date de sortie", "durée du film", "categories", "realisateurs","acteurs connue", "synopsys", 
          "Note Presee", "Note Publique", "url"]]
    while (ID != -1):
        page, url = get_next_page(nbr_of_page)
        all_presentations_of_this_page = get_all_presentation_of_this_page(page)
        ID, tab  = get_all_data_movie(all_presentations_of_this_page, ID, nbr_of_movie, url, False)
        all_tab = all_tab + tab
        nbr_of_page += 1
        if (nbr_of_page % 10 == 0):
            time.sleep(0.1)
        print("actuellement a la page = " + str(nbr_of_page - 1) + "| nome de film parcourue = " + str(ID))
    return all_tab
############################################################- afficher le tableau
def print_tab_data(tab_tab):
    print (tab_tab[0])
    for tab in tab_tab[1:]:
        affichage_data(None, None, None, None, None, None, None, None, None, None, None, None, tab)

## Fin de la premier partie 

A present j'ai fini mon code qui permet de recolet les informations, ils ont dans un tableau de tableau
pour recuper les infos de x nombre de film il suffit d'appeler la fonctionque j'ai crée

**get_all_data_from_movie**("Le nombre de film") -->
[['Id', 'titre', 'affiche', 'date de sortie', 'durée du film', 'categories', 'realisateurs', 'acteurs connue', 'synopsys', 'Note Presee', 'Note Publique', 'url'],
[(le tableau de chaque info de film)]]

In [34]:
#print_tab_data(get_all_data_from_movie(3))
tab = get_all_data_from_movie(10000)#l'Entête est compris dans le tableau
%clear
print("FIN")

actuellement a la page = 1| nome de film parcourue = 15
actuellement a la page = 2| nome de film parcourue = 30
actuellement a la page = 3| nome de film parcourue = 45
actuellement a la page = 4| nome de film parcourue = 60
actuellement a la page = 5| nome de film parcourue = 75
actuellement a la page = 6| nome de film parcourue = 90
actuellement a la page = 7| nome de film parcourue = 105
actuellement a la page = 8| nome de film parcourue = 120
actuellement a la page = 9| nome de film parcourue = 135
actuellement a la page = 10| nome de film parcourue = 150
actuellement a la page = 11| nome de film parcourue = 165
actuellement a la page = 12| nome de film parcourue = 180
actuellement a la page = 13| nome de film parcourue = 195
actuellement a la page = 14| nome de film parcourue = 210
actuellement a la page = 15| nome de film parcourue = 225
actuellement a la page = 16| nome de film parcourue = 240
actuellement a la page = 17| nome de film parcourue = 255
actuellement a la page = 18| 

---
# Etape 2 metre les infos Dans un csv

In [35]:
# Création du fichier CSV
with open('test.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(tab)

---
# Etape 3 go tout metre sur une vm et faire une Base de Données